In [ ]:
# For each question, predict the correctness of every student response as always either correct or incorrect, based on 
# whichever outcome is more common for that question in the training data.

import pandas as pd
import numpy as np
import argparse
import os
from scipy.sparse import csc_matrix

In [ ]:
base_dir = '../Data/'

input_dir = base_dir + "data/train_data"
train_data = base_dir + "data/train_data/train_task_1_2.csv"
test_data_1 = base_dir + "data/test_data/test_public_answers_task_1.csv"
# submission_template_dir = base_dir + "data/train_data"
# submission_template_file = base_dir + "data/train_data"
# output_dir = base_dir + "data/train_data"
# output_file = base_dir + "data/train_data"



In [ ]:
import pandas as pd
df = pd.read_csv(train_data)

In [ ]:
df.head()

In [ ]:
# df.groupby('UserId').sum()
df['UserId'].value_counts().describe()

In [ ]:
df['QuestionId'].value_counts().describe()

In [ ]:
df_test = pd.read_csv(test_data_1)

In [ ]:
df_test.head()

In [ ]:
tr_uid = set(df['UserId'].values)
test_uid = set(df_test['UserId'].values)

In [ ]:
tr_qid = set(df['QuestionId'].values)
test_qid = set(df_test['QuestionId'].values)

In [ ]:
# len(test_uid - tr_uid)
len(test_qid - tr_qid)

# Checks

In [ ]:
ques_ids = set(df['QuestionId'].values)
user_ids = set(df['UserId'].values)

In [ ]:
print(min(ques_ids))
print(max(ques_ids))
print(len(ques_ids))
print(min(user_ids))
print(max(user_ids))
print(len(user_ids))

In [ ]:
#TODO: Check if same QuestionID, UserId pair appears more than once

# Method 2

In [ ]:
mtr = df[['QuestionId','UserId','IsCorrect']]

In [ ]:
rows = df['UserId'].values
cols = df['QuestionId'].values
data = df['IsCorrect'].values
data = data*2-1

In [ ]:
ind = 4
print(rows[ind],cols[ind],data[ind])
print(mtr.values[ind])

In [ ]:
sparse_mtr = csc_matrix((data, (rows, cols)), dtype=np.int8)

In [ ]:
from sparsesvd import sparsesvd
K = 100
print(type(sparse_mtr))
U, s, Vt = sparsesvd(sparse_mtr, K)

In [ ]:
S = np.diag(np.sqrt(s))

In [ ]:
right_term = np.matmul(S,Vt)
right_term.shape
# U.shape

In [ ]:
def get_pred(qid,uid):
    user_features = U[:,uid]
    question_features = right_term[:,qid]
    score = np.dot(user_features,question_features)
    return np.sign(score)

In [ ]:
df_pred = df_test.copy()
print(df_pred.head())
df_pred['Prediction'] = df_pred.apply(lambda row: get_pred(row['QuestionId'],row['UserId']), axis=1)

In [ ]:
df_pred['Prediction'] = (df_pred['Prediction'] + 1)/2

In [ ]:
score_num = (df_pred['Prediction'] == df_pred['IsCorrect']).sum()
score_den = len(df_pred)
score = score_num / score_den
score